In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as F

spark = (
    SparkSession.builder.appName("pyspark_aula_21_criando_data_frame")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .getOrCreate()
)

compras = spark.read.parquet("./bases/COMPRAS.parquet").select('id','data','cd_livro','cd_cliente').alias('compras')
livros = spark.read.parquet("./bases/LIVROS.parquet").select('id', 'data_lancamento','preco').alias('livros')
autores = spark.read.parquet("./bases/AUTORES.parquet").alias('autores')

In [2]:
df = compras.join(livros, compras.cd_livro == livros.id).join(autores, livros.id == autores.id).drop('livros.id','autores.id')
df

id,data,cd_livro,cd_cliente,id,data_lancamento,preco,id,titulo,autor
012389,2021-07-24,030334762,3339828,030334762,2013-05-08,123.47,030334762,Em Busca do Tempo...,Marcel Proust
012534,2021-07-15,014347542,7799936,014347542,2007-04-05,182.43,014347542,Fogo Morto,José Lins do Rego
012574,2020-05-19,010325500,6273720,010325500,2000-09-09,30.62,010325500,A Obscena Senhora D,Hilda Hilst
012675,2021-07-11,035940339,649001,035940339,2014-08-11,213.6,035940339,Adeus às Armas,Ernest Hemingway
013457,2021-10-26,021006591,4312106,021006591,2019-02-24,96.84,021006591,Paulicéia Desvair...,Mário de Andrade
013478,2020-05-10,019488257,670483,019488257,2013-05-19,17.11,019488257,O Ódio que Você S...,Angie Thomas
013562,2021-10-28,035940339,1275633,035940339,2014-08-11,213.6,035940339,Adeus às Armas,Ernest Hemingway
013679,2020-10-19,030144651,1261140,030144651,2011-04-14,242.2,030144651,Zero,Ignácio De Loyola...
013687,2020-05-29,021923195,4098904,021923195,2015-02-22,38.89,021923195,O Ex-Mágico,Murilo Rubião
013796,2021-10-15,016778973,4024706,016778973,2017-06-23,99.81,016778973,Crime e Castigo,Fiódor Dostoiévski


In [9]:
window1 = Window.orderBy('compras.id')
window2 = Window.partitionBy('cd_cliente').orderBy('data')
window3 = Window.partitionBy('autor').orderBy('data_lancamento')

In [13]:
(
    df
    .withColumn('num_linha', F.row_number().over(window1))
    .withColumn('ordem_compra', F.row_number().over(window2))
    # .dropDuplicates(['cd_livro', 'autor'])
    # .withColumn('ordem_lancamento', F.row_number().over(window3))
    .withColumn('total_acumulado_cliente', F.round(F.sum('preco').over(window2),2))
)

id,data,cd_livro,cd_cliente,id,data_lancamento,preco,id,titulo,autor,num_linha,ordem_compra,total_acumulado_cliente
184365,2020-10-31,100520231,1010444,100520231,2009-08-17,35.55,100520231,Galáxias,Haroldo de Campos,3330,1,35.55
162408,2021-06-16,036059407,1010444,036059407,2021-01-20,222.98,036059407,As Histórias Comp...,Franz Kafka,2874,2,258.53
629783,2021-09-14,016581063,1010444,016581063,2013-05-16,145.25,016581063,Macunaíma – O Her...,Mário de Andrade,11521,3,403.78
594608,2021-10-11,030099528,1010444,030099528,2000-02-06,208.39,030099528,Retrato do Artist...,James Joyce,10824,4,612.17
561293,2021-11-02,030144651,1010444,030144651,2011-04-14,242.2,030144651,Zero,Ignácio De Loyola...,10118,5,854.37
648950,2022-01-15,022112497,1010444,022112497,2021-01-06,10.84,022112497,Mrs Dalloway,Virginia Woolf,11896,6,865.21
184567,2022-03-29,020414016,1010444,020414016,2011-06-09,212.1,020414016,Vidas Secas,Graciliano Ramos,3335,7,1077.31
026947,2022-05-04,036030824,1010444,036030824,2005-01-26,226.36,036030824,Tremor de Terra,Luiz Vilela,317,8,1303.67
472081,2022-06-19,026925428,1010444,026925428,2011-09-04,193.19,026925428,"Sing, Unburied, S...",Jesmyn Ward,8520,9,1496.86
027189,2022-06-26,012489208,1010444,012489208,2011-10-30,160.53,012489208,Triste Fim de Pol...,Lima Barreto,322,10,1657.39
